# Surface template_title validation using gridded observations

In [ ]:
chunk_start

In [ ]:
stamp = nc.session_info["stamp"]
out = ".trackers/" + stamp + ".txt"
if not os.path.exists(".trackers"):
    os.makedirs(".trackers")
# save out as empty file
with open(out, 'w') as f:
    f.write("")

In [ ]:
import numpy as np
grid = pd.read_csv("../../matched/model_grid.csv")
lon = grid.loc[:,[x for x in grid.columns if "lon" in x]].values
lon = np.unique(lon)
lon.sort()
lat = grid.loc[:,[x for x in grid.columns if "lat" in x]].values
lat = np.unique(lat)
lat.sort()
# get unique values in grid and sort them
lon = np.unique(lon)
lon.sort()
lon_res = lon[1] - lon[0]
lat_res = lat[1] - lat[0]

In [ ]:
variable = "template_variable" 
Variable = variable.title()
vv_name= variable
if vv_name == "co2flux":
    vv_name = "air-sea CO~2~ fluxes"
if vv_name == "poc":
    vv_name = "POC"
if vv_name == "doc":
    vv_name = "DOC"
if vv_name == "pco2":
    vv_name = "pCO2"
source = glob.glob(f"../../matched/gridded/**/**/**_{variable.lower()}*.nc")[0].split("/")[-1].split("_")[0]
domain = glob.glob(f"../../matched/gridded/**/**/**_{variable.lower()}*.nc")[0].split("/")[-3]

In [ ]:
# read in the dictionary of the variable matchup info
# this is a pickle
with open(f"../../matched/dicts/{domain}_{vv_name}_{source}_{variable}.pkl", 'rb') as f:
    vv_summary = pickle.load(f)

In [ ]:

if source == "nsbc":
    md("We used version 1.1 of the **North Sea Biogeochemical Climatology** (NSBC) to validate **sea surface template_variable**. NSBC is a monthly climatology that covers the region 47°-65°N and 15°W-15°E. The data is made up of observations over the period 1960-2014. For validation purposes we only used the level 3 data, which a gridded monthly climatology at a spatial resolution of 1/4°.  The data can be download from [NSBC](https://www.cen.uni-hamburg.de/en/icdc/data/ocean/nsbc.html).")
else:
    if variable == "poc":
        md("Model and observational POC were compared using data from the National Centre for Earth Observation (NCEO).")
        md("*Summary from NCEO*")
        md("The National Centre for Earth Observation (NCEO): Monthly global Particulate Organic Carbon (POC) dataset contains POC concentrations gridded on both sinusoidal (SIN) and geographic (GEO) grid projections at 4 km spatial resolution for 1997-2020. The POC dataset has been produced using the Ocean Colour Climate Change Initiative Remote Sensing Reflectance (Rrs) products, Version 4.2. The dataset includes the Rrs at 443 nm and 555 nm with pixel-by-pixel uncertainty estimates for each wavelength.")
        md("For more details on the algorithm and its validation, please see papers by Stramski et al. (2008) and Evers-King et al. (2017). Please note that the validation of the POC algorithm is a continuing process. To increase the accuracy of POC algorithms, further in situ POC data need to be collected with high spatial and temporal resolution.")

    if variable == "doc":
        md("## Data Source: NCEO BICEP project DOC")
        md("Modelled DOC does not included the refactory component, which is typically 40 uM. This was added to the model data to make it comparable to the NCEO data.")
    
    if source == "ostia":
        md("Temperature was validated using the OSTIA sea surface temperature dataset. The validation was performed by comparing the modelled temperature with the OSTIA data for the same time and location. The OSTIA data was downloaded from the Copernicus Marine Environment Monitoring Service (CMEMS) catalogue. A description of the dataset is available [here](https://data.marine.copernicus.eu/product/SST_GLO_SST_L4_REP_OBSERVATIONS_010_011/description).")
    
    if source == "cobe2":
        md(f"Temperature was validated using the COBE2 sea surface temperature dataset. The validation was performed by comparing the modelled temperature with the COBE2 data for the same time and location. The COBE2 data was downloaded from https://psl.noaa.gov/data/gridded/data.cobe2.html.")
        md(f"Observational temperature is a monthly time series from 1850 with a spatial resolution of 1°x1°.")

md(f"**Matchup procedure**: The model and observations were matched up as follows. First, the model dataset was cropped by a small amount to make sure cells close to the boundary were removed. The model was then regridded to the observational grid if the observational grid was coarser using nearest neighbour. Only grid cells with model and observational data were maintained.")

df_mapping = pd.read_csv("../../matched/mapping.csv")
model_variable = list(df_mapping.query("variable == @variable").model_variable)[0]

md(f"The following model output was used to compare with observational **{vv_name}**: **{model_variable}**.")

if source == "woa":
    md("Monthly surface data was extracted from the 2023 version of the NOAA World Ocean Atlas (WOA) dataset. The WOA dataset is a set of objectively analysed climatological fields of in situ oceanographic observations. The data is available at a spatial resolution of 1°x1°. The data can be downloaded from [NOAA](https://www.nodc.noaa.gov/OC5/woa18/).")

In [ ]:
ff = glob.glob(f"../../matched/gridded/**/**/**_{variable.lower()}_*surface.nc")
if len(ff) != 1:
    raise ValueError("Something is wrong with the file")
layer = os.path.basename(ff[0]).split("_")[-1].replace(".nc", "")
ds_model = nc.open_data(ff)
ds_model.set_precision("F32")
ds_model.subset(variable = "model")
ds_model.tmean("month")
ds_year = min(ds_model.years)
ds_model.set_year(ds_year)
ds_times = ds_model.times
df_times = pd.DataFrame({"year":[x.year for x in ds_times]}).groupby("year").size().reset_index()
df_times.columns = ["year", "count"]
years = list(df_times.query("count > 1").year)
ds_model.as_missing(0)
# if variable is doc, add 40
ds_model.run()
ds_annual = ds_model.copy()
ds_annual.tmean()
# ds_annual.set_longnames({ds_annual.variables[0]: Variable})

## Baseline climatologies of surface template_title

In [ ]:
md(f"Climatologies of model and observational {layer} {vv_name} are shown in the figures below.")
# model climatology years can be derived from the vv_summary
try:
    clim_years = vv_summary["clim_years"]
    md(f"The model climatology is calculated using the years **{clim_years[0]}-{clim_years[-1]}**.") 
except:
    pass

In [ ]:
ff = glob.glob(f"../../matched/gridded/**/**/**_{variable}_*surface.nc")
ds_obs = nc.open_data(ff)
ds_obs.subset(variable = "observation")
ds_obs.set_precision("F32")
ds_obs.tmean("month")
ds_obs.run()

In [ ]:
## fix the units and names

vars = [
        "ammonium",
        "chlorophyll",
        "nitrate",
        "phosphate",
        "oxygen",
        "silicate",
        "poc",
        "doc",
    ]
if variable in vars:
    # set the units of obs to match model
    ds_obs.set_units({ds_obs.variables[0]: ds_model.contents.unit[0]})
    if variable not in ["poc", "doc"]:
        # set the longnames of obs to match model
        ds_obs.set_longnames({ds_obs.variables[0]: f"Observed surface {vv_name} concentration"})
        ds_model.set_longnames({ds_model.variables[0]: f"Modelled surface {vv_name} concentration"})
        ds_annual.set_longnames({ds_annual.variables[0]: f"Modelled annual mean surface {variable} concentration"})
    else:
        ds_obs.set_longnames({ds_obs.variables[0]: f"Observed surface {variable.upper()} concentration"})
        ds_model.set_longnames({ds_model.variables[0]: f"Modelled surface {variable.upper()} concentration"})
        ds_annual.set_longnames({ds_annual.variables[0]: f"Modelled annual mean surface {variable.upper()} concentration"})
if variable == "temperature":
    ds_obs.set_longnames({ds_obs.variables[0]: "Observed SST"})
    ds_model.set_longnames({ds_model.variables[0]: "Modelled SST"})
    ds_annual.set_longnames({ds_annual.variables[0]: "Modelled SST"})

ds_obs.run()
ds_model.run()


In [ ]:
chunk_clim

In [ ]:
chunk_bias

In [ ]:
md(f"## Can the model reproduce seasonality of {layer} {vv_name}?")

md(f"The ability of the model to reproduce seasonality of {layer} {vv_name} was assessed by comparing the modelled and observed seasonal cycle of {vv_name}. First, we derive a monthly climatology for the model data. Then, we calculate the Pearson correlation coefficient between the modelled and observed {vv_name} at each grid cell. The Pearson correlation coefficient is a measure of the linear correlation between two variables. It has a value between -1 and 1, where 1 indicates a perfect positive linear correlation, 0 indicates no linear correlation, and -1 indicates a perfect negative linear correlation.")


md("Note: we are only assessing the ability of the model to reproduce the ability of the model to reproduce seasonal changes, not long-term trends.")


In [ ]:
chunk_seasonal

In [ ]:
if regional:
    md(f"## Regional assessment of model performance for {layer} {vv_name}")

In [ ]:
if regional:
    if not global_grid:
        md("We assessed the regional performance of the model by comparing the model with observations from the following regions: Southern North Sea, Central North Sea, Northern North Sea, Channel, Skagerrak, Norwegian Trench, Shetland, Irish Shelf, Irish Sea, Celtic Sea, Armorican, Northern North East Atlantic, Southern North East Atlantic, Shelf, Ocean, Rosa, Locate Shelf, Deep Ocean.")
    else:
        md("We assessed the regional performance of the model by comparing the model with observations from the following regions: Arctic Ocean, Indian Ocean, North Atlantic, South Atlantic, North Pacific, South Pacific, and Southern Ocean.") 

    md("The regions considered are mapped below.")

In [ ]:
if regional:
    lon_name = [x for x in ds_regions.to_xarray().coords if "lon" in x][0]
    lat_name = [x for x in ds_regions.to_xarray().coords if "lat" in x][0]
    df_mapped = (
        ds_regions
        .to_dataframe()
        .reset_index()
        # rename the columns
        .rename(columns = {lon_name: "lon", lat_name: "lat"})
        .melt(id_vars = ["lon", "lat"])
        .dropna()
        .merge(regions_contents.loc[:,["variable", "long_name"]])
        .drop(columns = [ "value"])
    )
    bad = ["Rosa", "Locate Shelf"]
    df_mapped = df_mapped.query("long_name not in @bad")
    xlim = np.array([df_mapped.lon.min(), df_mapped.lon.max()])
    ylim = np.array([df_mapped.lat.min(), df_mapped.lat.max()])

    def fix_name(x):
        x = x.replace("North East", "NE")
        x = x.replace("North ", "N ")
        if x == "Channel":
            x = "English Channel"
        return x

    fix_name = np.vectorize(fix_name)


    df_mapped.long_name = fix_name(df_mapped.long_name)

else:
    df_mapped = 1




In [ ]:
if regional:
    df_all = []
    df_summary = []
    for vv in ds_regions.variables:
        ds_rr = ds_regions.copy()
        ds_rr.subset(variable = vv)
        ds_rr.run()
        ds_vv = ds_ts.copy()
        time_name = [x for x in list(ds_vv.to_xarray().coords) if "time" in x][0]
        ds_vv * ds_rr
        ds_region = ds_vv.copy()
        ds_cor = ds_vv.copy()
        ds_cor.tmean()
        ds_cor.cor_space("model", "observation")
        region = list(regions_contents.query("variable == @vv").long_name)[0]
        df1 = (
            ds_cor
            .to_dataframe()
            .reset_index()
            .loc[:,["cor"]]
            .rename(columns = {"cor": "Spatial correlation"})
            .assign(region = region)
        )
        # now do the temporal correlation

        ds_cor = ds_vv.copy()
        ds_cor.cor_time("model", "observation")
        ds_cor.spatial_mean()
        df2 = (
            ds_cor
            .to_dataframe()
            .reset_index()
            .loc[:,["cor"]]
            .rename(columns = {"cor": "Temporal correlation"})
            .assign(region = region)
        )
        df = df1.merge(df2)

        ds_vv.spatial_mean()
        df_bias = (
            ds_vv
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .assign(month = lambda x: x.time.dt.month)
            .assign(bias = lambda x: x.model - x.observation)
            .assign(region = region)
            .groupby("region")
            .mean()
            .reset_index()
            .loc[:,["region", "bias"]]
        )
        df = df1.merge(df2).merge(df_bias)
        df_summary.append(df)
        df_vv = (
            ds_vv
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .melt("time")
            .assign(month = lambda x: x.time.dt.month)
            .assign(region = vv)
        )
        df_all.append(df_vv)
        ds_region.tmean()
        df_region = (
            ds_region
            .to_dataframe()
            .dropna()
            .reset_index()
            .loc[:,["model", "observation"]]
            .drop_duplicates()
        )
    
        del ds_rr, ds_vv, ds_region
    df_all = pd.concat(df_all).dropna()
        
    df_all = (
        df_all
        .merge(df_mapped.loc[:,["long_name", "variable"]].drop_duplicates().rename(columns = {"variable": "region"}))
    )
    df_summary= pd.concat(df_summary)

In [ ]:
# restrict df_mapped to regions in df_all
if regional:
    regions = set(df_all.query("value > 0").region)
    df_mapped = df_mapped.query("variable in @regions")

In [ ]:
%%capture --no-display
%%R -i regional -i df_mapped -i xlim -i ylim -i global_grid

if (regional){

    library(tidyverse)

    world_map <- map_data("world")

    gg <-  ggplot(df_mapped)+
        geom_tile(aes(x = lon, y = lat))+
        coord_cartesian(xlim = xlim, ylim = ylim)+
        theme_bw(base_size = 10)+
        facet_wrap(~long_name)+
        theme(axis.title.x = element_blank(),
              axis.title.y = element_blank())

y_labels <-  as.numeric(na.omit(layer_scales(gg)$y$break_positions()))
x_labels <- as.numeric(na.omit(layer_scales(gg)$x$break_positions()))
x_breaks <- x_labels
y_breaks <- y_labels

# y labels are north-south coordinates. Make them more appropriate
# i.e. 10 should be 10 °N, -10 should be 10 °S

y_labels <- ifelse(y_labels >= 0, paste0(y_labels, "°N"), paste0(abs(y_labels), "°S"))
x_labels <- ifelse(x_labels >= 0, paste0(x_labels, "°E"), paste0(abs(x_labels), "°W"))

gg <- gg + scale_y_continuous(breaks = y_breaks, labels = y_labels)+
    scale_x_continuous(breaks = x_breaks, labels = x_labels)+
        geom_polygon(data = world_map, aes(x = long, y = lat, group = group), fill = "grey", color = "grey")



    gg

}

In [ ]:
if regional:
    md(f"**Figure {chapter}{i_figure}**: Regions used for validation of {layer} {vv_name}.")
i_figure += 1

In [ ]:
if regional:
    md(f"Time series were constructed comparing the monthly mean of the spatial average {layer} {vv_name} in each region. The spatial average was calculated using the mean of all grid cells within each region, accounting for grid cell area.")

In [ ]:
if regional:
    ylab = "Spatial average " + vv_name + " ("+ nc.static_plot.fix_label(ds_ts.contents.unit[0]) + ")"
    gg = (
        ggplot(df_all)+
        geom_line(aes("month", "value", colour = "variable"))+
        facet_wrap("long_name")+
        labs(y = ylab )+
        labs(x = "Month")+
        theme(legend_position = "top")+
        scale_color_manual(values = ["red", "blue"])+
        theme_bw(base_size = 10)+
        labs(colour = "")+
        scale_x_continuous(breaks = [1,4, 7, 10], labels = ["Jan", "Apr", "Jul", "Oct"]) +
        theme(legend_position = "top") 
        
    )
    
    gg = gg.draw()
    gg

    out_ts = f"../../results/regionals/{source}_{variable}_regionals.csv"
    # check if directory exists for out_ts
    if not os.path.exists("../../results/regionals"):
        os.makedirs("../../results/regionals")
    df_all.to_csv(out_ts, index = False)


In [ ]:
if regional:
    md(f"**Figure {chapter}{i_figure}**: Seasonal cycle of {layer} {vv_name} for model and observations for each region. The spatial average is taken over the region.") 
    i_figure += 1

In [ ]:
if regional:
    md(f"The table below shows the average bias of surface {vv_name} in each region. The bias is calculated as the modelled value minus the observed value. A positive bias indicates that the model overestimates the observed value, while a negative bias indicates that the model underestimates the observed value.")

if regional:
    # output and hide index
    # first average things and tidy
    df_summary = (
        df_summary
        # .loc[:,["region", "bias"]]
        .groupby("region")
        .mean()
        .reset_index()
        # capitalize the columns
        .rename(columns = {"region": "Region", "bias": "Bias"})
    )
    if not global_grid:
        # drop the spatial correlation
        df_summary = df_summary.drop(columns = ["Spatial correlation"])
    df_summary.style.hide(axis="index")

In [ ]:
if regional:
    regional_summary = [f"**Table {chapter}{i_figure}**: Summary of performance of the model surface {vv_name} in each region."]
    regional_summary.append("The bias column represents the spatial average of the annual mean modelled value minus the observed value.")
    regional_summary.append("The temporal correlation column represents the spatial mean of the temporal correlation between the model and observations per grid cell.") 
    if global_grid:
        regional_summary.append("The spatial correlation column represents the spatial correlation between the model and observations.")
    regional_summary = " ".join(regional_summary).replace("  ", " ")
    md(regional_summary)

In [ ]:
chunk_results

In [ ]:
md(f"## Can the model reproduce spatial patterns of {layer} {vv_name}?")

md(f"The ability of the model to reproduce spatial patterns of {layer} {vv_name} was assessed by comparing the modelled and observed {vv_name} at each grid cell. We calculated the Pearson correlation coefficient between the modelled and observed {vv_name} at each grid cell. The Pearson correlation coefficient is a measure of the linear correlation between two variables. It has a value between -1 and 1, where 1 indicates a perfect positive linear correlation, 0 indicates no linear correlation, and -1 indicates a perfect negative linear correlation.")
md("This was carried out monthly and using the annual mean in each grid cell")

In [ ]:
ff = glob.glob(f"../../matched/gridded/**/**/**_{variable}*surface.nc")
if len(ff) != 1:
    raise ValueError("Something is wrong with the file")
layer = os.path.basename(ff[0]).split("_")[-1].replace(".nc", "")
ds_cor = nc.open_data(ff)
ds_cor.set_precision("F32")
ds_cor.tmean("month")
ds_cor.cor_space("model", "observation")
ds_cor_df = ds_cor.to_dataframe().reset_index()
ds_cor_df = ds_cor_df.dropna()
time_name = [x for x in list(ds_cor.to_xarray().coords) if "time" in x][0]
# rename time in dataframe
ds_cor_df.rename(columns = {time_name: "time"}, inplace = True)
# extract the month
ds_cor_df["month"] = ds_cor_df.time.dt.month
ds_cor_df = ds_cor_df.loc[:,["month", "cor"]].drop_duplicates()
# change month number to month name
ds_cor_df["month"] = ds_cor_df["month"].apply(lambda x: calendar.month_abbr[x])
# now do this annually
ds_cor = nc.open_data(ff)
ds_cor.set_precision("F32")
ds_cor.tmean("month")
ds_cor.tmean()
ds_cor.cor_space("model", "observation")
ds_cor_df_annual = ds_cor.to_dataframe().reset_index()
ds_cor_df_annual = ds_cor_df_annual.dropna()
time_name = [x for x in list(ds_cor.to_xarray().coords) if "time" in x][0]
# rename time in dataframe
ds_cor_df_annual.rename(columns = {time_name: "time"}, inplace = True)
# extract the month
ds_cor_df_annual["month"] = ds_cor_df_annual.time.dt.month
ds_cor_df_annual = ds_cor_df_annual.loc[:,["month", "cor"]].drop_duplicates()
# output to csv
ds_cor_df_annual = ds_cor_df_annual.assign(month = "Annual mean")
# merge the two dataframes
ds_cor_df = pd.concat([ds_cor_df_annual, ds_cor_df])
# change month to period
ds_cor_df.rename(columns = {"month": "period"}, inplace = True)
# Give the columns more sensible names
ds_cor_df.rename(columns = {"cor": "Correlation coefficient"}, inplace = True)
ds_cor_df.rename(columns = {"period": "Time period"}, inplace = True)
# drop any na
ds_cor_df.dropna(inplace = True)
ds_cor_df.style.hide(axis="index")

In [ ]:
md(f"**Table {chapter}{i_table}**: Pearson correlation coefficient between modelled and observed {layer} {vv_name} at each grid cell. The correlation was calculated monthly and using the annual mean in each grid cell.")
i_table += 1

In [ ]:
chunk_cdf

In [ ]:
if global_grid:
    md(f"## Hovmöller diagram of {layer} {vv_name} for the Atlantic and Pacific Oceans")
else:
    if source == "woa":
        md(f"## Hovmöller diagram of {layer} {vv_name} for the full domain") 

In [ ]:
hov_plot = False
df_hovs = False
if global_grid or source == "woa":
    # find the vertical paths
    paths = glob.glob(f"../../matched/gridded/**/**/**_{variable}*surface.nc")
    df_hovs = []
    if len(paths) == 1:
        for rr in ["atlantic", "pacific"]:
            ds = ds_model.copy() 
            ds.append(ds_obs)
            ds.merge("variables", "month")
            if global_grid:
                ds_rr = ds_regions.copy()
                ds_rr.subset(variable = rr)
                ds.regrid(ds_rr)
                ds * ds_rr
            ds.zonal_mean()

            lat_name = [x for x in list(ds.to_xarray().coords) if "lat" in x][0]
            time_name = [x for x in list(ds.to_xarray().coords) if "time" in x][0]
            rr_region = rr
            if not global_grid:
                rr_region = "Full domain"
            df = (
                ds
                .to_dataframe()
                .dropna()
                .reset_index()
                .loc[:,[lat_name, time_name, "model", "observation"]]
                .rename(columns = {lat_name: "lat", time_name: "time"})
                # get the month
                .assign(month = lambda x: x.time.dt.month)
                # drop time
                .drop(columns = "time")
                .assign(region = rr_region) 

            )
            df_hovs.append(df)
            if not global_grid:
                break
        df_hovs = pd.concat(df_hovs)
        hov_plot = True

In [ ]:
%%R -i df_hovs -i vv_name -i model_unit -i global_grid -i source -i hov_plot -w 800  -h 600 
if(hov_plot){
library(tidyverse)
# hovmoller plot

df_hovs <- df_hovs %>%
    gather("variable", "value", "model", "observation") %>%
    mutate(variable = ifelse(variable == "model", "Model", "Observation"))

if (vv_name != "Temperature")
    vv_name <- paste(vv_name, "concentration", sep = " ")
vv_name <- str_to_title(vv_name)
# add model_unit in brackets
vv_name <- paste(vv_name, "(", model_unit, ")", sep = "")

gg <- df_hovs %>%
    # turn region into title
    mutate(region = str_to_title(region)) %>%
    ggplot(aes(x = month, y = lat, fill = value))+
    geom_tile()+
    theme_bw(base_size = 18)+
    facet_grid(region~variable)+
    # do month as J F M etc. Only first letter of month
    scale_x_continuous(breaks = 1:12, labels = month.abb, expand = c(0, 0))+
    theme(
    legend.position = "bottom", legend.direction = "horizontal", legend.box = "horizontal", legend.key.width = unit(2.0, "cm"),
    legend.key.height = unit(0.3, "cm"))+
    scale_fill_viridis_c(na.value = "white",
                       #breaks = c(0.4, 0.6, 0.8, 1.0), labels = c("0.4", "0.6", "0.8", ">1"),
                       guide = guide_colorbar(title.position = "bottom", title.hjust = 0.5, title.theme = element_text(angle = 0, size = 16 , family = "Helvetica"))
  )+
    labs(fill = vv_name)+
    # remove x and y labs
    theme(axis.title.x = element_blank(),
            axis.title.y = element_blank())
y_labels <-  as.numeric(na.omit(layer_scales(gg)$y$break_positions()))
y_breaks <- y_labels

y_labels <- ifelse(y_labels >= 0, paste0(y_labels, "°N"), paste0(abs(y_labels), "°S"))

gg <- gg +
    scale_y_continuous(breaks = y_breaks, labels = y_labels)
gg

}

In [ ]:
if global_grid:
    md(f"**Figure {chapter}{i_figure}**: Hovmöller diagram of {layer} {vv_name} for the Atlantic and Pacific Oceans. The diagram shows the zonal mean of {vv_name} concentration for the model and observations.")
    i_figure += 1
else:
    if source == "woa":
        md(f"**Figure {chapter}{i_figure}**: Hovmöller diagram of {layer} {vv_name} for the full domain. The diagram shows the zonal mean of {vv_name} concentration for the model and observations.") 
        i_figure += 1

In [ ]:
if global_grid:
    md(f"## Can the model reproduce spatial and vertical patterns in {vv_name}?")
    md(f"The ability of the model to reproduce broadscale patterns in the vertical distribution of {vv_name} was assessed by comparing the zonal averages in the model and WOA23 data set for the Atlantic, Pacific, and Indian Oceans. The zonal averages were calculated using the annual average.") 
    md(f"Figure {chapter}{i_figure} shows the zonal averages.") 
else:
    if source == "woa":
        md(f"## Can the model reproduce spatial and vertical patterns in {vv_name}?")
        md(f"The ability of the model to reproduce broadscale patterns in the vertical distribution of {vv_name} was assessed by comparing the zonal averages in the model and WOA23 data set for the full domain. The zonal averages were calculated using the annual average.") 
        md(f"Figure {chapter}{i_figure} shows the zonal averages.")

In [ ]:
zonal_plot = False
if global_grid or source == "woa":
    # find the vertical paths
    paths = glob.glob(f"../../matched/gridded/**/**/**_{variable}*vertical.nc")
    df_zonals = []
    if len(paths) == 1:
        for rr in ["atlantic", "pacific"]:
            ds = nc.open_data(paths)
            if global_grid:
                ds_rr = ds_regions.copy()
                ds_rr.subset(variable = rr)
                ds.regrid(ds_rr)
                ds * ds_rr
            ds.tmean("year")
            ds.zonal_mean()

            df1 = (
                ds.to_xarray()["model"]
                .to_dataframe()
                .reset_index()
                .dropna()
            )
            depth_name = [x for x in  ds.to_xarray()["model"].coords if "depth" in x][0]
            lon_name = [x for x in  ds.to_xarray()["model"].coords if "lon" in x][0]
            lat_name = [x for x in  ds.to_xarray()["model"].coords if "lat" in x][0]
            #     .drop_duplicates()
            #     # change lat_name to lat
            #     .rename(columns = {lat_name: "lat"})
            #     # change depth_name to depth
            #     .rename(columns = {depth_name: "depth"})
            #     # melt model and observation
            #     .melt(id_vars = ["lat", "depth"])
            #     .assign(region = rr.title())
            # )
            df1 = (
                df1
                .loc[:,[lat_name, depth_name, "model"]]
                .rename(columns = {lat_name: "lat", depth_name: "depth"})
                #.assign(region = rr.title())
            )

            df2 = (
                ds.to_xarray()["observation"]
                .to_dataframe()
                .reset_index()
                .dropna()
            )

            depth_name = [x for x in  ds.to_xarray()["observation"].coords if "depth" in x][0]
            lon_name = [x for x in  ds.to_xarray()["observation"].coords if "lon" in x][0]
            lat_name = [x for x in  ds.to_xarray()["observation"].coords if "lat" in x][0]
            rr_region = rr
            if not global_grid:
                rr_region = "full domain"

            df2 = (
                df2
                .loc[:,[lat_name, depth_name, "observation"]]
                .rename(columns = {lat_name: "lat", depth_name: "depth"})
            #    .assign(region = rr.title())
            )
            df = df1.merge(df2) 
            df = (
                df
                .melt(id_vars = ["lat", "depth"])
                .assign(region = rr_region.title())
            )
            
              #  .loc[:,[lat_name, depth_name, "model", "observation"]]
            df_zonals.append(df)
            if not global_grid:
                break
        df_zonals = pd.concat(df_zonals)
        # output to adhoc folder
        out_file = "adhoc/df_zonals.feather"
        # create folder if it doesn't exist
        if not os.path.exists("adhoc"):
            os.makedirs("adhoc")
        df_zonals.to_feather(out_file)
        zonal_plot = True

In [ ]:
%%R -i global_grid -i model_unit -i vv_name -i source -i zonal_plot
library(tidyverse)
if (zonal_plot){
    df_zonals <- arrow::read_feather("adhoc/df_zonals.feather")
df_zonals <- df_zonals %>%
    mutate(depth = -depth) %>%
    mutate(depth = depth / 1000) %>%
    arrange(desc(depth)) %>%
    group_by(region,lat, variable) %>%
    mutate(lag = lag(depth)) %>%
    replace_na(list(lag = 0)) %>%
    mutate(width = depth - lag)

# make vv_name a title
# if vv_name is not temperature add concentration
if (vv_name != "Temperature")
    vv_name <- paste(vv_name, "concentration", sep = " ")
vv_name <- str_to_title(vv_name)
# add model_unit in brackets
vv_name <- paste(vv_name, "(", model_unit, ")", sep = "")

gg <- df_zonals %>%
    # turn variable into title
    mutate(variable = str_to_title(variable)) %>%
    ggplot()+
    geom_tile(aes(lat, depth, fill = value, height = width))+
    theme_bw(base_size = 16)+
    facet_grid(region~variable)+
    # coord_flip()+
    labs(y = "Depth (km)", x = "Latitude")+
    # x scale between -90 and 90, provide suitable labels with 30 split
    theme(
    legend.position = "bottom", legend.direction = "horizontal", legend.box = "horizontal", legend.key.width = unit(2.0, "cm"),
    legend.key.height = unit(0.3, "cm"))+
    scale_fill_viridis_c(na.value = "white",
                       #breaks = c(0.4, 0.6, 0.8, 1.0), labels = c("0.4", "0.6", "0.8", ">1"),
                       guide = guide_colorbar(title.position = "bottom", title.hjust = 0.5, title.theme = element_text(angle = 0, size = 12, family = "Helvetica"))
  )+
    labs(fill = vv_name)+
    theme(axis.title.x = element_blank())
    # get rid of whitespace

    # put the legend

x_labels <- as.numeric(na.omit(layer_scales(gg)$x$break_positions()))
x_breaks <- x_labels

x_labels <- ifelse(x_labels >= 0, paste0(x_labels, "°N"), paste0(abs(x_labels), "°S"))

gg <- gg +
    scale_x_continuous(breaks = x_breaks, labels = x_labels)
gg
    

}

In [ ]:
if zonal_plot:
    if global_grid:
        md(f"**Figure {chapter}{i_figure}**: Zonal mean of {vv_name} for model and observations for the Atlantic and Pacific Oceans. The depth is given in km.")
        i_figure += 1
    else:
        if source == "woa":
            md(f"**Figure {chapter}{i_figure}**: Zonal mean of {vv_name} for model and observations for the full domain. The depth is given in km.")
            i_figure += 1

In [ ]:
%%R -i global_grid -i model_unit -i vv_name -i zonal_plot
library(tidyverse)
if (zonal_plot){
    df_zonals <- arrow::read_feather("adhoc/df_zonals.feather")
# now do a bias plot

df_zonals <- df_zonals %>%
    select(lat, region, depth, variable, value) %>%
    spread(variable, value) %>%
    mutate(bias = model - observation)
df_zonals <- df_zonals %>%
    mutate(depth = -depth) %>%
    mutate(depth = depth / 1000) %>%
    arrange(desc(depth)) %>%
    group_by(region,lat) %>%
    mutate(lag = lag(depth)) %>%
    replace_na(list(lag = 0)) %>%
    mutate(width = depth - lag)

# make vv_name a title
# if vv_name is not temperature add concentration
if (vv_name != "Temperature")
    vv_name <- paste(vv_name, "concentration", sep = " ")
vv_name <- str_to_title(vv_name)
# add model_unit in brackets
vv_name <- paste(vv_name, "(", model_unit, ")", sep = "")
# add bias to the start
vv_name <- paste("Bias in", vv_name)

df_zonals %>%
    # turn variable into title
    ggplot()+
    geom_tile(aes(lat, depth, fill = bias, height = width))+
    theme_bw(base_size = 16)+
    facet_wrap(~region)+
    # coord_flip()+
    labs(y = "Depth (km)", x = "Latitude")+
    # x scale between -90 and 90, provide suitable labels with 30 split
    scale_x_continuous(breaks = seq(-90, 60, 30), labels = c("90°S", "60°S", "30°S", "0°", "30°N", "60°N"), limits = c(-90, 90), expand = c(0,0))+
    scale_y_continuous(expand = c(0,0))+
    theme(
    legend.position = "bottom", legend.direction = "horizontal", legend.box = "horizontal", legend.key.width = unit(2.0, "cm"),
    legend.key.height = unit(0.3, "cm"))+
    # scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0,
                    #    guide = guide_colorbar(title.position = "bottom", title.hjust = 0.5, title.theme = element_text(angle = 0, size = 12, family = "Helvetica"))+
    labs(fill = vv_name)+
    scale_fill_gradient2(low = "blue", high = "red", guide = guide_colourbar(title.position = "bottom", title.hjust = 0.5, title.theme = element_text(angle = 0, size = 12, family = "Helvetica")))+
        theme(axis.title.x = element_blank())
    # get rid of whitespace

    # put the legend
    

}

In [ ]:
if zonal_plot:
    if global_grid:
        md(f"**Figure {chapter}{i_figure}**: Bias in zonal mean of {vv_name} for model and observations for the Atlantic and Pacific Oceans. The depth is given in km.")
        i_figure += 1

In [ ]:
time_series = False
if regional:
    ff = glob.glob(f"../../matched/gridded/**/**/**_{variable}*surface.nc")
    ds_ts = nc.open_data(ff)
    years = ds_ts.years
    year_range = f"{min(years)}-{max(years)}"
    if len(years) > 1:
        mask_all(ds_ts)
        ds_ts.tmean("year")
        ds_ts.run()
        time_series = True

In [ ]:
if time_series:
    md(f"The ability of the model to reproduce mult-year trends in {layer} {vv_name} was assessed by comparing the modelled and observed time series of annual {vv_name} across each region.")
    md(f"The figure below shows the average {vv_name} in each region")

In [ ]:
if time_series:
    df_all = []
    for vv in ds_regions.variables:
        ds_rr = ds_regions.copy()
        ds_rr.subset(variable = vv)
        ds_rr.run()
        ds_vv = ds_ts.copy()
        ds_vv * ds_rr
        ds_region = ds_vv.copy()
        ds_vv.spatial_mean()
        region = list(regions_contents.query("variable == @vv").long_name)[0]
        time_name = [x for x in list(ds_vv.to_xarray().coords) if "time" in x][0]
        df_vv = (
            ds_vv
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .melt("time")
            .assign(year = lambda x: x.time.dt.year)
            .assign(region = vv)
        )
        df_all.append(df_vv)
        ds_region.tmean()
        df_region = (
            ds_region
            .to_dataframe()
            .dropna()
            .reset_index()
            .loc[:,["model", "observation"]]
            .drop_duplicates()
        )
    
        del ds_rr, ds_vv, ds_region
    df_all = pd.concat(df_all).dropna()
        
    df_all = (
        df_all
        .merge(df_mapped.loc[:,["long_name", "variable"]].drop_duplicates().rename(columns = {"variable": "region"}))
    )

In [ ]:
if time_series:
    # save df_all to csv
    ylab = "Spatial average " + variable + " ("+ nc.static_plot.fix_label(ds_ts.contents.unit[0]) + ")"
    
    gg = (
        ggplot(df_all)+
        geom_line(aes("year", "value", colour = "variable"))+
        facet_wrap("long_name")+
        labs(y = ylab )+
        labs(x = "Year")+
        theme(legend_position = "top")+
        scale_color_manual(values = ["red", "blue"])+
        theme_bw(base_size = 10)+
        labs(colour = "")+
        theme(legend_position = "top") 
        
    )
    
    gg = gg.draw()
    gg


In [ ]:
if time_series:
    md(f"**Figure {chapter}{i_figure}**: Changes in {layer} {vv_name} for model and observations for each region for the period {year_range}. The spatial average is taken over the region.") 
    i_figure += 1

In [ ]:
chunk_end

## Data Sources

In [ ]:
if source == "nsbc":
    md("Hinrichs,Iris; Gouretski,Viktor; Paetsch,Johannes; Emeis, Kay; Stammer, Detlef (2017). North Sea Biogeochemical Climatology (Version 1.1).")
    md("URL: https://www.cen.uni-hamburg.de/en/icdc/data/ocean/nsbc.html")
if variable == "poc":
    md("Sathyendranath, S.; Kong, C.; Jackson, T. (2021): NCEO: Monthly global Particulate Organic Carbon (POC) (produced from the Ocean Colour Climate Change Initiative, Version 4.2 dataset). Centre for Environmental Data Analysis, 07 January 2021. doi:10.5285/ef09d81517a84979ac60329e4859f449. https://dx.doi.org/10.5285/ef09d81517a84979ac60329e4859f449")
    md("URL: https://catalogue.ceda.ac.uk/uuid/ef09d81517a84979ac60329e4859f449")

if source == "ostia":
    md("Good, S.; Fiedler, E.; Mao, C.; Martin, M.J.; Maycock, A.; Reid, R.; Roberts-Jones, J.; Searle, T.; Waters, J.; While, J.; Worsfold, M. The Current Configuration of the OSTIA System for Operational Production of Foundation Sea Surface Temperature and Ice Concentration Analyses. Remote Sens. 2020, 12, 720, doi:10.3390/rs12040720")
    md("URL: https://data.marine.copernicus.eu/product/SST_GLO_SST_L4_REP_OBSERVATIONS_010_011/description")

if source == "cobe2":
    md("COBE-SST 2 and Sea Ice data provided by the NOAA PSL, Boulder, Colorado, USA, from their website at https://psl.noaa.gov.")

if source == "woa":
    md("Reagan, James R.; Boyer, Tim P.; García, Hernán E.; Locarnini, Ricardo A.; Baranova, Olga K.; Bouchard, Courtney; Cross, Scott L.; Mishonov, Alexey V.; Paver, Christopher R.; Seidov, Dan; Wang, Zhankun; Dukhovskoy, Dmitry (2023). World Ocean Atlas 2023 (NCEI Accession 0270533). [indicate subset used]. NOAA National Centers for Environmental Information. Dataset. https://www.ncei.noaa.gov/archive/accession/0270533. Accessed 13/05/2024.")

if global_grid:
    md("Regional subdomains were based on mapped regions from the Marine Regions database. The Marine Regions database is available at http://www.marineregions.org.")



